In [1]:
import pandas as pd
import pyodbc

In [2]:
def conectar_base_datos():
    SERVER = 'ACHEPE'
    DATABASE = 'elecciones-IA-data'

    connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;'

    conn = pyodbc.connect(connectionString) 
    return conn


In [3]:
def pasar_a_DF(dat_cursor):
    columnas = dat_cursor.description
    nombre = []
    datos = []

    for i in range(len(columnas)):
        nombre.append(columnas[i][0])
        datos.append([])

    for fila in dat_cursor:
        for i in range(len(columnas)):
            datos[i].append(fila[i])
    
    diccionario = dict()
    for i in range(len(columnas)):
        diccionario[nombre[i]] = datos[i]

    df = pd.DataFrame(diccionario)

    return df

In [4]:
def Consultar_datos_tabla(tabla):
        conn = conectar_base_datos()
        cursor = conn.cursor()
        SQL_STATEMENT = f"""
                SELECT *
                FROM {tabla}
                """
        records = cursor.execute(SQL_STATEMENT)
        
        return records

In [7]:
tablas = ['Candidatos','Fuentes','Partidos','Publicaciones','Publicaciones_politicas']

In [10]:
for t in tablas:
    consulta = Consultar_datos_tabla(t)
    df = pasar_a_DF(consulta)
    df.to_csv(f'C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/data_base/data_base_csv/{t}.csv')

In [9]:
consulta_publicaciones_join_politicas = """
                                    SELECT * 
                                    FROM Publicaciones_politicas
                                    LEFT JOIN Publicaciones
                                    ON Publicaciones_politicas.id = Publicaciones.id
                                        """

In [10]:
conn = conectar_base_datos()
cursor = conn.cursor()
records = cursor.execute(consulta_publicaciones_join_politicas)

In [11]:
df_join =pasar_a_DF(records)

In [14]:
df_join.drop('cant_likes',axis=1,inplace=True)

In [15]:
df_join.to_csv('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/data_base/data_base_csv/Publicaciones_JOIN_politicas.csv')